In [14]:
import pandas as pd
import io
import os

from google.cloud import storage

from pyspark.sql import DataFrame, SparkSession 
from pyspark.sql.functions import *
from pyspark.sql.types import *

from unidecode import unidecode



In [15]:
import os
credentials_path = "/home/jovyan/code/.devcontainer/soulcode-iam-key.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

Na GCP crie uma bucket (Cloud Storage) com qualquer nome e dentro dela crie uma pasta com o nome de dados_bruto e dentro dele salve o arquivo (csv) com os dados do da base escolhida no site https://dados.gov.br/, deixe o acesso público ativado

In [16]:
BUCKET_NAME =  'soulcode-analytics-data-bootcamp'
NAME_FILE = '/content/lista-de-bases.csv'
DEST_BLOB_NAME = 'raw_data/lista-de-bases.csv'
DEST_BLOB_NAME_PROCESSED = 'clean_data/lista-de-dados-processed.csv'

In [17]:
# Upload raw data

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )
# upload_blob(BUCKET_NAME, NAME_FILE, DEST_BLOB_NAME)

No colab faça a leitura desse arquivo e a partir dele crie diferentes insight (pandas/pyspark/numpy) com os resultados de agrupamentos e salve esses arquivos como csv e envie diretamente pelo cada um deles para o bucket dentro de uma pasta chamada "Dados_tratados"

In [31]:
def download_blob_into_memory(bucket_name, blob_name):
    """Downloads a blob into memory."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your GCS object
    # blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    # Construct a client side representation of a blob.
    # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
    # any content from Google Cloud Storage. As we don't need additional data,
    # using `Bucket.blob` is preferred here.
    blob = bucket.blob(blob_name)
    contents = blob.download_as_bytes()

    print(
        "Downloaded storage object {} from bucket {} as the following bytes object: {}.".format(
            blob_name, bucket_name, contents.decode("utf-8")
        )
    )
    return contents

data = download_blob_into_memory(BUCKET_NAME, DEST_BLOB_NAME)
df = pd.read_csv(io.BytesIO(data))


Downloaded storage object raw_data/lista-de-bases.csv from bucket soulcode-analytics-data-bootcamp as the following bytes object: sg_base,ds_base,Órgão - id_orgao → nm_ugd,sg_rename_periodo,tp_informacao_publica,tp_informacao_sigiliosa,sg_periodo_atualizacao,Distinct-Id
Acervo educacional sobre água - ConhecerRH,Repositório de conteúdos educacionais em recursos hídricos,Agência Nacional de Águas e Saneamento Básico,MENSAL,SIM,NAO,ME,1
Acompanhamento das Cotas de Importação,"Relaciona as cotas tarifárias de importação vigentes, com informações sobre a utilização de cada produto contingenciado.",Ministério da Economia,MENSAL,SIM,NAO,ME,1
Alterações Orçamentárias (créditos),"Contém dados dos orçamentos fiscal e da seguridade social, relacionados aos créditos adicionais e às alterações orçamentárias, compreendidos em um exercício fiscal.",Ministério da Economia,ANUAL,SIM,NAO,AN,1
Aplicativo Água e Esgotos - Água e Esgotos,Aplicativo que disponibiliza os dados e informações do projeto Atlas

In [32]:
df.head()

,sg_base,ds_base,Órgão - id_orgao → nm_ugd,sg_rename_periodo,tp_informacao_publica,tp_informacao_sigiliosa,sg_periodo_atualizacao,Distinct-Id
0,Acervo educacional sobre água - ConhecerRH,Repositório de conteúdos educacionais em recur...,Agência Nacional de Águas e Saneamento Básico,MENSAL,SIM,NAO,ME,1
1,Acompanhamento das Cotas de Importação,Relaciona as cotas tarifárias de importação vi...,Ministério da Economia,MENSAL,SIM,NAO,ME,1
2,Alterações Orçamentárias (créditos),Contém dados dos orçamentos fiscal e da seguri...,Ministério da Economia,ANUAL,SIM,NAO,AN,1
3,Aplicativo Água e Esgotos - Água e Esgotos,Aplicativo que disponibiliza os dados e inform...,Agência Nacional de Águas e Saneamento Básico,ANUAL,SIM,NAO,AN,1
4,Aplicativo Declara Água - Declara Água,O Declara Água é uma ferramenta simples para q...,Agência Nacional de Águas e Saneamento Básico,MENSAL,NAO,NAO,ME,1


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   sg_base                    304 non-null    object
 1   ds_base                    304 non-null    object
 2   Órgão - id_orgao → nm_ugd  304 non-null    object
 3   sg_rename_periodo          304 non-null    object
 4   tp_informacao_publica      304 non-null    object
 5   tp_informacao_sigiliosa    304 non-null    object
 6   sg_periodo_atualizacao     304 non-null    object
 7   Distinct-Id                304 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 19.1+ KB


In [34]:
 df.describe(include='all')

,sg_base,ds_base,Órgão - id_orgao → nm_ugd,sg_rename_periodo,tp_informacao_publica,tp_informacao_sigiliosa,sg_periodo_atualizacao,Distinct-Id
count,304,304,304,304,304,304,304,304.0
unique,304,303,22,7,2,2,7,NaN
top,Acervo educacional sobre água - ConhecerRH,Base de dados do oriunda do DW - PGFN,Agência Nacional de Energia Elétrica,DIÁRIA,SIM,NAO,DI,NaN
freq,1,2,67,120,251,196,120,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [35]:
def clean_column_names(df):

    if isinstance(df, pd.DataFrame):
        # For Pandas DataFrames
        df.columns = df.columns.map(lambda col: unidecode(col.lower().replace(' ', '_').replace('-', '')))
        df.columns = df.columns.str.replace('__', '_')
        print(pd.DataFrame)
    else:
        # For PySpark DataFrames
        for col in df.columns:
            df = df.withColumnRenamed(col, unidecode(col.lower().replace(' ', '_').replace('-', '')))
            print(df)
    return df


In [37]:
df = clean_column_names(df)

<class 'pandas.core.frame.DataFrame'>


In [38]:
df = df.drop(columns=['distinctid'])

In [40]:
data_processed = '../dataset/lista-de-bases_processed.csv'
df.to_csv(data_processed, index=False)

In [42]:
upload_blob(BUCKET_NAME, data_processed, DEST_BLOB_NAME_PROCESSED)

File ../dataset/lista-de-bases_processed.csv uploaded to clean_data/lista-de-dados-processed.csv.


- https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python (upload)
- https://cloud.google.com/storage/docs/downloading-objects (download)
- https://cloud.google.com/sdk/docs/install#deb
